# Klasifikasi scene akustik perkotaan dan upaya untuk melihat persamaan dan perbedaan dari masing-masing kernel

Muhammad Farhan Alharits
09011281823067
SK5B

Sumber : https://github.com/ManushiM/UrbanSoundScenes_SupportVectorMachines

In [1]:
#mengimpor library

import numpy as np
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
from sklearn import svm
rcParams['figure.figsize'] = 8, 6

In [2]:
#baca dataset

train = pd.read_csv('Data1_train.csv')
test = pd.read_csv('Data1_test.csv')
train.head()

,mean0,mean1,mean2,mean3,mean4,mean5,mean6,mean7,mean8,mean9,...,var_diff11,var_diff12,var_diff13,var_diff14,var_diff15,var_diff16,var_diff17,var_diff18,var_diff19,scenes
0,0.575796,0.717401,0.539698,-1.062370,0.191049,-1.723536,-1.194319,0.053656,-0.209370,-0.200844,...,-0.655437,-0.839057,-0.300258,-0.830228,-0.783522,-0.736217,-0.683096,-0.866754,-0.571898,tubestation
1,-0.626746,1.382491,0.447212,-1.766357,0.009479,2.536803,0.380988,1.120369,3.892846,4.094601,...,-0.906597,-0.561817,-0.990822,-0.910512,-0.770320,-1.019891,-1.231433,-0.857304,-0.888658,train-ter
2,-0.484043,0.597128,1.028187,-1.412017,1.534679,0.511047,1.286969,0.967864,-0.259907,0.790211,...,1.939481,2.260830,2.335424,2.172651,2.198847,2.151799,1.910007,1.362777,1.451639,bus
3,0.308431,-0.005826,-0.932353,0.143836,-0.619640,0.029328,-0.851747,-0.587147,0.131899,-0.957469,...,0.320351,0.050667,-0.057665,0.239617,0.260419,0.668990,0.290236,0.469636,0.520772,market
4,-1.846697,-1.260525,1.779260,0.112545,0.020945,0.628017,0.523070,1.356682,1.079687,0.648798,...,3.062474,0.293887,3.250679,-0.146490,2.591120,-0.208007,0.856651,0.327243,-0.246140,train-ter


In [3]:
#Membagi untuk mengambil 10 fitur pertama saja

trainSet = train.iloc[:,:11]
trainSet.head()

,mean0,mean1,mean2,mean3,mean4,mean5,mean6,mean7,mean8,mean9,mean10
0,0.575796,0.717401,0.539698,-1.062370,0.191049,-1.723536,-1.194319,0.053656,-0.209370,-0.200844,-2.482419
1,-0.626746,1.382491,0.447212,-1.766357,0.009479,2.536803,0.380988,1.120369,3.892846,4.094601,2.210399
2,-0.484043,0.597128,1.028187,-1.412017,1.534679,0.511047,1.286969,0.967864,-0.259907,0.790211,-1.639074
3,0.308431,-0.005826,-0.932353,0.143836,-0.619640,0.029328,-0.851747,-0.587147,0.131899,-0.957469,0.435051
4,-1.846697,-1.260525,1.779260,0.112545,0.020945,0.628017,0.523070,1.356682,1.079687,0.648798,0.379519


In [4]:
# Mendapatkan bagian Y yang sesuai (teksnya)

Y_labels = train.scenes
Y_labels[:15]

0     tubestation
1       train-ter
2             bus
3          market
4       train-ter
5     tubestation
6       train-ter
7             bus
8      restaurant
9      busystreet
10     busystreet
11     busystreet
12    tubestation
13     busystreet
14    tubestation
Name: scenes, dtype: object

In [5]:
#Fungsi untuk kasih angka ke kategori

def numericLabels(x):
     return {
        ourLabels[0]: 1,
        ourLabels[1]: 2,
        ourLabels[2]: 3,
        ourLabels[3]: 4,
        ourLabels[4]: 5,
        'unknown': 6,
    }[x]

In [6]:
#Fungsi untuk kasih nilai numerik ke label
ourLabels = ['tubestation', 'quietstreet', 'busystreet', 'restaurant', 'openairmarket']

def manageLabels(labelsText, labelsNum):
    i = 0;
    while i < labelsText.size:
        if labelsText[i] not in ourLabels:
            labelsText.replace(labelsText[i],'unknown',inplace=True)
        labelsNum[i] = numericLabels(labelsText[i])
        i += 1


In [7]:
#Membuat label berdasarkan apa yang udah ditentukan

Y_train = Y_labels

#Manggil fungsi manageLabels diatas
manageLabels(Y_labels, Y_train)

#konvert seri baru jadi int
Y_train = Y_train.astype('int64')    
print (Y_train[:20])

C:\Users\falha\anaconda3\envs\python3.7\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


0     1
1     6
2     6
3     6
4     6
5     1
6     6
7     6
8     4
9     3
10    3
11    3
12    1
13    3
14    1
15    6
16    3
17    2
18    1
19    3
Name: scenes, dtype: int64


### Ulangi untuk data pengujian

In [8]:
#Membagi untuk mengambil 10 fitur pertama saja
testSet = test.iloc[:,:11]
testSet.head()

,mean0,mean1,mean2,mean3,mean4,mean5,mean6,mean7,mean8,mean9,mean10
0,0.659110,0.259060,0.848907,1.232900,0.690433,0.814238,2.048807,0.163775,-0.359654,1.258600,0.305429
1,0.887682,1.164037,0.694511,0.719179,0.184981,-0.047482,1.817482,-0.463925,-0.948119,1.184027,0.139446
2,1.246803,-0.326879,0.009353,0.755241,1.341334,-0.898195,-0.117060,0.497354,0.580539,0.811553,-1.326550
3,1.155296,-0.048561,0.222940,0.901589,0.344102,0.504171,1.625438,-1.127245,-0.891104,1.743355,0.591964
4,0.369011,0.651106,1.115642,-0.791181,2.553240,-1.560783,-0.523261,1.630105,1.417551,0.693929,-1.228802


In [9]:
# Mendapatkan bagian Y yang sesuai (teksnya)

Y_labelsT = test.scenes
Y_labelsT[:15]

0            bus
1            bus
2            bus
3            bus
4            bus
5            bus
6            bus
7            bus
8            bus
9            bus
10    busystreet
11    busystreet
12    busystreet
13    busystreet
14    busystreet
Name: scenes, dtype: object

In [10]:
#Membuat label berdasarkan apa yang udah ditentukan

Y_test = Y_labelsT

#manggil fungsi
manageLabels(Y_labelsT, Y_test)

#konvert seri baru ke int
Y_test = Y_test.astype('int64')    
print (Y_test[:20])

0     6
1     6
2     6
3     6
4     6
5     6
6     6
7     6
8     6
9     6
10    3
11    3
12    3
13    3
14    3
15    3
16    3
17    3
18    3
19    3
Name: scenes, dtype: int64


C:\Users\falha\anaconda3\envs\python3.7\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


### Training the model

In [11]:
# ngelatih modelnya (Poly derajatnya=2)
for kernel in ('linear', 'poly', 'rbf'):
    clf = svm.SVC(kernel=kernel, C=0.7, degree=2)
    clf.fit(trainSet, Y_train)
    print ("Berhasil memprediksi {0} persen dari data {1} kernel untuk training data".format(100-abs(clf.predict(trainSet)-Y_train).sum()/len(Y_train), kernel))

Berhasil memprediksi 99.2609090909091 persen dari data linear kernel untuk training data
Berhasil memprediksi 99.22818181818182 persen dari data poly kernel untuk training data
Berhasil memprediksi 99.74454545454546 persen dari data rbf kernel untuk training data


### Testing the model

In [12]:
#mencocokkan modelnya (Poly derajatnya=2, C=0.5)
for kernel in ('linear', 'poly', 'rbf'):
    clf = svm.SVC(kernel=kernel, C=0.5, degree=2)
    clf.fit(testSet, Y_test)
    correct=1.0*(clf.predict(testSet)==np.asarray(Y_test)).sum()/len(Y_test)
    print ("Berhasil memprediksi {0} persen dari data {1} kernel untuk training data".format((correct)*100, kernel))

Berhasil memprediksi 69.0 persen dari data linear kernel untuk training data
Berhasil memprediksi 56.00000000000001 persen dari data poly kernel untuk training data
Berhasil memprediksi 73.0 persen dari data rbf kernel untuk training data


In [13]:
#mencocokkan modelnya (Poly derajatnya=2, C=0.6)
for kernel in ('linear', 'poly', 'rbf'):
    clf = svm.SVC(kernel=kernel, C=0.6, degree=2)
    clf.fit(testSet, Y_test)
    correct=1.0*(clf.predict(testSet)==np.asarray(Y_test)).sum()/len(Y_test)
    print ("Berhasil memprediksi {0} persen dari data {1} kernel untuk training data".format((correct)*100, kernel))

Berhasil memprediksi 70.0 persen dari data linear kernel untuk training data
Berhasil memprediksi 56.99999999999999 persen dari data poly kernel untuk training data
Berhasil memprediksi 75.5 persen dari data rbf kernel untuk training data


In [14]:
#mencocokkan modelnya (Poly derajatnya=2, C=0.7)
for kernel in ('linear', 'poly', 'rbf'):
    clf = svm.SVC(kernel=kernel, C=0.7, degree=2)
    clf.fit(testSet, Y_test)
    correct=1.0*(clf.predict(testSet)==np.asarray(Y_test)).sum()/len(Y_test)
    print ("Berhasil memprediksi {0} persen dari data {1} kernel untuk training data".format((correct)*100, kernel))

Berhasil memprediksi 70.5 persen dari data linear kernel untuk training data
Berhasil memprediksi 57.99999999999999 persen dari data poly kernel untuk training data
Berhasil memprediksi 77.0 persen dari data rbf kernel untuk training data
